In [30]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
import xgboost as xgb

In [32]:
path=Path('/kaggle/data_science_bowl')
path

PosixPath('/kaggle/data_science_bowl')

### Read Data

In [33]:
def read_data():
    train_df = pd.read_csv(path/'train.csv')
    test_df = pd.read_csv(path/'test.csv')
    train_labels_df = pd.read_csv(path/'train_labels.csv')
    sample_submission_df = pd.read_csv(path/'sample_submission.csv')
    return train_df, test_df, train_labels_df, sample_submission_df

In [34]:
train_df, test_df, train_labels_df, sample_submission_df = read_data()

In [35]:
train_df.head()

,event_id,game_session,timestamp,event_data,installation_id,event_count,event_code,game_time,title,type,world
0,27253bdc,45bb1e1b6b50c07b,2019-09-06T17:53:46.937Z,"{""event_code"": 2000, ""event_count"": 1}",0001e90f,1,2000,0,Welcome to Lost Lagoon!,Clip,NONE
1,27253bdc,17eeb7f223665f53,2019-09-06T17:54:17.519Z,"{""event_code"": 2000, ""event_count"": 1}",0001e90f,1,2000,0,Magma Peak - Level 1,Clip,MAGMAPEAK
2,77261ab5,0848ef14a8dc6892,2019-09-06T17:54:56.302Z,"{""version"":""1.0"",""event_count"":1,""game_time"":0...",0001e90f,1,2000,0,Sandcastle Builder (Activity),Activity,MAGMAPEAK
3,b2dba42b,0848ef14a8dc6892,2019-09-06T17:54:56.387Z,"{""description"":""Let's build a sandcastle! Firs...",0001e90f,2,3010,53,Sandcastle Builder (Activity),Activity,MAGMAPEAK
4,1bb5fbdb,0848ef14a8dc6892,2019-09-06T17:55:03.253Z,"{""description"":""Let's build a sandcastle! Firs...",0001e90f,3,3110,6972,Sandcastle Builder (Activity),Activity,MAGMAPEAK


In [36]:
test_df.head()

,event_id,game_session,timestamp,event_data,installation_id,event_count,event_code,game_time,title,type,world
0,27253bdc,0ea9ecc81a565215,2019-09-10T16:50:24.910Z,"{""event_code"": 2000, ""event_count"": 1}",00abaee7,1,2000,0,Welcome to Lost Lagoon!,Clip,NONE
1,27253bdc,c1ea43d8b8261d27,2019-09-10T16:50:55.503Z,"{""event_code"": 2000, ""event_count"": 1}",00abaee7,1,2000,0,Magma Peak - Level 1,Clip,MAGMAPEAK
2,27253bdc,7ed86c6b72e725e2,2019-09-10T16:51:51.805Z,"{""event_code"": 2000, ""event_count"": 1}",00abaee7,1,2000,0,Magma Peak - Level 2,Clip,MAGMAPEAK
3,27253bdc,7e516ace50e7fe67,2019-09-10T16:53:12.825Z,"{""event_code"": 2000, ""event_count"": 1}",00abaee7,1,2000,0,Crystal Caves - Level 1,Clip,CRYSTALCAVES
4,7d093bf9,a022c3f60ba547e7,2019-09-10T16:54:12.115Z,"{""version"":""1.0"",""round"":0,""event_count"":1,""ga...",00abaee7,1,2000,0,Chow Time,Game,CRYSTALCAVES


In [37]:
train_labels_df.head()

,game_session,installation_id,title,num_correct,num_incorrect,accuracy,accuracy_group
0,6bdf9623adc94d89,0006a69f,Mushroom Sorter (Assessment),1,0,1.0,3
1,77b8ee947eb84b4e,0006a69f,Bird Measurer (Assessment),0,11,0.0,0
2,901acc108f55a5a1,0006a69f,Mushroom Sorter (Assessment),1,0,1.0,3
3,9501794defd84e4d,0006a69f,Mushroom Sorter (Assessment),1,1,0.5,2
4,a9ef3ecb3d1acc6a,0006a69f,Bird Measurer (Assessment),1,0,1.0,3


## Feature Engineering

In [38]:
main_key = 'installation_id'

In [48]:
def get_event_counts(df, column:str='event_id', simple_event_count=True):
    df_event_count = df.groupby([main_key, column]).agg({column: ['count']}) \
    if simple_event_count else df.groupby([main_key, column]).agg({'event_count': 'sum'})
    df_event_count.columns = ['count']
    df_event_count_pivot = df_event_count.pivot_table(index = main_key, columns = [column], values = 'count')
    return df_event_count_pivot.fillna(0)

In [51]:
get_event_counts(train_df, column='event_id', simple_event_count=False)

event_id,003cd2ee,0086365d,00c73085,01ca3a3c,022b4259,02a42007,0330ab6a,0413e89d,04df9b66,05ad839b,...,f56e0afc,f5b8c21a,f6947f54,f71c4741,f7e47413,f806dc10,f93fc684,fbaf3456,fcfdffb6,fd20ea40
installation_id,,,,,,,,,,,,,,,,,,,,,
0001e90f,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,24357.0,24453.0,311.0,0.0,0.0,0.0,0.0
000447c4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,139.0,0.0,0.0,0.0,0.0
0006a69f,0.0,0.0,0.0,0.0,2132.0,6200.0,0.0,1454.0,0.0,0.0,...,2.0,191.0,48.0,3144.0,2907.0,400.0,0.0,523.0,3517.0,0.0
0006c192,0.0,2.0,366.0,0.0,547.0,0.0,363.0,734.0,0.0,0.0,...,1.0,0.0,36.0,1698.0,1493.0,0.0,2.0,1287.0,3816.0,0.0
0009a5a9,0.0,0.0,957.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
fff64664,0.0,0.0,0.0,0.0,0.0,0.0,8612.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1018.0,1042.0,0.0,4.0,0.0,0.0,0.0
fff70c1e,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,183.0,194.0,0.0,0.0,0.0,0.0,0.0
fff744ce,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [50]:
get_event_counts(train_df, simple_event_count=True)

event_id,003cd2ee,0086365d,00c73085,01ca3a3c,022b4259,02a42007,0330ab6a,0413e89d,04df9b66,05ad839b,...,f56e0afc,f5b8c21a,f6947f54,f71c4741,f7e47413,f806dc10,f93fc684,fbaf3456,fcfdffb6,fd20ea40
installation_id,,,,,,,,,,,,,,,,,,,,,
0001e90f,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,66.0,66.0,4.0,0.0,0.0,0.0,0.0
000447c4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0
0006a69f,0.0,0.0,0.0,0.0,24.0,70.0,0.0,29.0,0.0,0.0,...,2.0,3.0,2.0,48.0,45.0,9.0,0.0,18.0,45.0,0.0
0006c192,0.0,1.0,3.0,0.0,8.0,0.0,9.0,10.0,0.0,0.0,...,1.0,0.0,1.0,15.0,14.0,0.0,1.0,14.0,35.0,0.0
0009a5a9,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
fff64664,0.0,0.0,0.0,0.0,0.0,0.0,57.0,0.0,0.0,0.0,...,0.0,0.0,0.0,17.0,17.0,0.0,2.0,0.0,0.0,0.0
fff70c1e,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,10.0,10.0,0.0,0.0,0.0,0.0,0.0
fff744ce,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [52]:
agg_stats = ['mean', 'sum', 'min', 'max', 'std', 'skew', 'median']
main_key = 'installation_id'

def get_object_columns(df, columns):
    df = df.groupby([main_key, columns])['event_id'].count().reset_index()
    df = df.pivot_table(index = main_key, columns = [columns], values = 'event_id')
    df.columns = list(df.columns)
    df.fillna(0, inplace = True)
    return df

def get_numeric_columns(df, column):
    df = df.groupby(main_key).agg({f'{column}': agg_stats})
    df[column].fillna(df[column].mean(), inplace = True)
    df.columns = [f'{column}_{stat}' for stat in agg_stats]
    return df

def get_numeric_columns_add(df, agg_column, column):
    df = df.groupby([main_key, agg_column]).agg({f'{column}': agg_stats}).reset_index()
    df = df.pivot_table(index = main_key, columns = [agg_column], values = [col for col in df.columns if col not in [main_key, agg_column]])
    df[column].fillna(df[column].mean(), inplace = True)
    df.columns = list(df.columns)
    return df

def feature_engineering(train_df, test_df, train_labels_df):
    
    numerical_columns = ['game_time']
    categorical_columns = ['type', 'world']

    comp_train_df = pd.DataFrame({main_key: train_df[main_key].unique()})
    comp_train_df.set_index(main_key, inplace = True)
    comp_test_df = pd.DataFrame({main_key: test_df[main_key].unique()})
    comp_test_df.set_index(main_key, inplace = True)
    
    merge_args = {'left_index':True, 'right_index':True}

    for i in numerical_columns:
        comp_train_df = comp_train_df.merge(get_numeric_columns(train_df, i), **merge_args)
        comp_test_df = comp_test_df.merge(get_numeric_columns(test_df, i), **merge_args)
    
    for i in categorical_columns:
        comp_train_df = comp_train_df.merge(get_object_columns(train_df, i), **merge_args)
        comp_test_df = comp_test_df.merge(get_object_columns(test_df, i), **merge_args)
    
    for i in categorical_columns:
        for j in numerical_columns:
            comp_train_df = comp_train_df.merge(get_numeric_columns_add(train_df, i, j), **merge_args)
            comp_test_df = comp_test_df.merge(get_numeric_columns_add(test_df, i, j), **merge_args)
            
    comp_train_df = comp_train_df.merge(get_event_counts(train_df, column='event_id', simple_event_count=True), **merge_args)
    comp_test_df = comp_test_df.merge(get_event_counts(test_df, column='event_id', simple_event_count=True), **merge_args)
    
    comp_train_df = comp_train_df.merge(get_event_counts(train_df, column='event_code', simple_event_count=True), **merge_args)
    comp_test_df = comp_test_df.merge(get_event_counts(test_df, column='event_code', simple_event_count=True), **merge_args)
    
    comp_train_df.reset_index(inplace = True)
    comp_test_df.reset_index(inplace = True)
    
    print(f'Our training set has {comp_train_df.shape[0]} rows and {comp_train_df.shape[1]} columns')

    # get the mode of the title
    labels_map = dict(train_labels_df.groupby('title')['accuracy_group'].agg(lambda x:x.value_counts().index[0]))
    # merge target
    labels = train_labels_df[[main_key, 'title', 'accuracy_group']]
    # replace title with the mode
    labels.loc[:,'title'] = labels['title'].map(labels_map)
    # get title from the test set
    comp_test_df.loc[:,'title'] = test_df.groupby(main_key).last()['title'].map(labels_map).reset_index(drop = True)
    # join train with labels
    comp_train_df = labels.merge(comp_train_df, on = main_key, how = 'left')
    print(f'We have {comp_train_df.shape[0]} training rows')
    
    return comp_train_df, comp_test_df

In [53]:
comp_train_df, comp_test_df = feature_engineering(train_df, test_df, train_labels_df)

Our training set has 17000 rows and 498 columns


/opt/anaconda3/envs/deeplearning/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


We have 17690 training rows


In [54]:
pd.options.display.max_columns = None
comp_test_df

,installation_id,game_time_mean,game_time_sum,game_time_min,game_time_max,game_time_std,game_time_skew,game_time_median,Activity,Assessment,Clip,Game,CRYSTALCAVES,MAGMAPEAK,NONE,TREETOPCITY,"(game_time, max, Activity)","(game_time, max, Assessment)","(game_time, max, Clip)","(game_time, max, Game)","(game_time, mean, Activity)","(game_time, mean, Assessment)","(game_time, mean, Clip)","(game_time, mean, Game)","(game_time, median, Activity)","(game_time, median, Assessment)","(game_time, median, Clip)","(game_time, median, Game)","(game_time, min, Activity)","(game_time, min, Assessment)","(game_time, min, Clip)","(game_time, min, Game)","(game_time, skew, Activity)","(game_time, skew, Assessment)","(game_time, skew, Clip)","(game_time, skew, Game)","(game_time, std, Activity)","(game_time, std, Assessment)","(game_time, std, Clip)","(game_time, std, Game)","(game_time, sum, Activity)","(game_time, sum, Assessment)","(game_time, sum, Clip)","(game_time, sum, Game)","(game_time, max, CRYSTALCAVES)","(game_time, max, MAGMAPEAK)","(game_time, max, NONE)","(game_time, max, TREETOPCITY)","(game_time, mean, CRYSTALCAVES)","(game_time, mean, MAGMAPEAK)","(game_time, mean, NONE)","(game_time, mean, TREETOPCITY)","(game_time, median, CRYSTALCAVES)","(game_time, median, MAGMAPEAK)","(game_time, median, NONE)","(game_time, median, TREETOPCITY)","(game_time, min, CRYSTALCAVES)","(game_time, min, MAGMAPEAK)","(game_time, min, NONE)","(game_time, min, TREETOPCITY)","(game_time, skew, CRYSTALCAVES)","(game_time, skew, MAGMAPEAK)","(game_time, skew, NONE)","(game_time, skew, TREETOPCITY)","(game_time, std, CRYSTALCAVES)","(game_time, std, MAGMAPEAK)","(game_time, std, NONE)","(game_time, std, TREETOPCITY)","(game_time, sum, CRYSTALCAVES)","(game_time, sum, MAGMAPEAK)","(game_time, sum, NONE)","(game_time, sum, TREETOPCITY)",0086365d,00c73085,022b4259,02a42007,0330ab6a,0413e89d,04df9b66,05ad839b,06372577,070a5291,08fd73f3,08ff79ad,0a08139c,0d18d96c,0d1da71f,0db6d71d,1325467d,1340b8d7,1375ccb7,14de4c5d,155f62a4,1575e76c,15a43e5b,15ba1109,15eb4a7d,15f99afc,160654fd,16667cc5,16dffff1,17113b36,19967db1,1996c610,1af8be29,1bb5fbdb,1beb320a,1c178d24,1cc7cfca,1cf54632,1f19558b,222660ff,2230fab4,250513af,25fa8af4,262136f4,26a5a3dd,26fd2d99,27253bdc,28520915,28a4eb9a,28ed704e,28f975ea,29bdd9ba,29f54413,2a444e03,2a512369,2b058fe3,2b9272f4,2c4e6db0,2dc29e21,2dcad279,2fb91ec1,30614231,30df3273,31973d56,3323d7e9,33505eae,3393b68b,363c86c9,363d3849,36fa3ebe,37937459,37c53127,37db1c2f,37ee8496,38074c54,392e14df,3a4be871,3afb49e6,3afde5dd,3b2048ee,3babcb9b,3bb91ced,3bb91dda,3bf1cf26,3bfd1a65,3ccd3f02,3d0b9317,3d63345e,3d8c61b0,3dcdda7f,3ddc79c3,3dfd4aa4,3edf6747,3ee399c3,44cb4907,45d01abe,461eace6,46b50ba8,46cd75b4,47026d5f,47efca07,47f43a44,48349b14,4901243f,499edb7c,49ed92e9,4a09ace1,4a4c3d21,4b5efe37,4bb2f698,4c2ec19f,4d6737eb,4d911100,4e5fc6f5,4ef8cdd3,51102b85,51311d7a,5154fc30,5290eab1,532a2afb,5348fd84,53c6e11a,55115cbd,562cec5f,565a3990,56817e2b,56bcd38d,56cd3b43,5859dfb6,587b5989,58a0de5c,598f4598,5a848010,5b49460a,5be391b5,5c2f29ca,5c3d2b2f,5d042115,5de79a6a,5e109ec3,5e3ea25a,5e812b27,5f0eb72c,5f5b2617,6043a2b4,6077cc36,6088b756,63f13dd7,65a38bf7,65abac75,67439901,67aa2ada,69fdac0a,6aeafed4,6bf9e3e1,6c517a88,6c930e6e,6cf7d25c,6d90d394,6f445b57,6f4adc4b,6f4bd64e,6f8106d9,7040c096,709b1251,71e712d8,71fe8f75,731c0cbe,736f9581,7372e1a5,73757a5e,7423acbc,74e5f8a7,7525289a,756e5507,763fc34e,76babcde,77261ab5,77c76bc5,77ead60d,792530f8,795e4a37,7961e599,7ab78247,7ad3efc6,7cf1bc53,7d093bf9,7d5c30a2,7da34a02,7dfe6d8a,7ec0c298,7f0836bf,804ee27f,828e68f9,832735e1,83c6c409,84538528,84b0e0c8,857f21c0,85d1b0de,85de926c,86ba578b,86c924c4,87d743c1,884228c8,88d4a5be,895865f3,89aace00,8ac7cce4,8af75982,8b757ab8,8d748b58,8d7e386c,8d84fa81,8f094001,8fee50e2,907a054b,90d848e0,90ea0bac,90efca10,91561152,923afab1,92687c59,93b353f2,93edfe2e,9554a50b,99abe2bb,99ea62f3,9b01374f,9b23e8ee,9b4001e4,9c5ef70c,9ce586dd,9d29771f,9d4e7b25,9de5e594,9e34ea74,9e4c8c7b,9e6b7fb5,9ed8f6da,

## Normalize

In [55]:
# comp_train_df.columns = [c if type(c) != tuple else '_'.join(c) for c in comp_train_df.columns]

In [56]:
# comp_test_df.columns = [c if type(c) != tuple else '_'.join(c) for c in comp_test_df.columns]

In [57]:
# list(comp_train_df.columns)

In [58]:
# def normalize(df: pd.DataFrame, cont_names):
#     "Compute the means and stds of `self.cont_names` columns to normalize them."
#     means, stds = {},{}
#     for n in cont_names:
#         means[n], stds[n] = df[n].mean(), df[n].std()
#         df[n] = (df[n]-means[n]) / (1e-7 + stds[n])

In [59]:
# normalize(comp_train_df, [c for c in comp_train_df.columns if c not in ['installation_id', 'title', 'accuracy_group']])
# normalize(comp_test_df, [c for c in comp_test_df.columns if c not in ['installation_id', 'title', 'accuracy_group']])

## Training

In [60]:
# quadratic weighted kappa
def qwk3(a1, a2, max_rat=3):
    '''
    a1 - ground truth
    a2 - predicted values
    '''
    assert(len(a1) == len(a2))
    a1 = np.asarray(a1, dtype=int)
    a2 = np.asarray(a2, dtype=int)
    hist1 = np.zeros((max_rat + 1, ))
    hist2 = np.zeros((max_rat + 1, ))
    o = 0
    for k in range(a1.shape[0]):
        i, j = a1[k], a2[k]
        hist1[i] += 1
        hist2[j] += 1
        o +=  (i - j) * (i - j)
    e = 0
    for i in range(max_rat + 1):
        for j in range(max_rat + 1):
            e += hist1[i] * hist2[j] * (i - j) * (i - j)
    e = e / a1.shape[0]
    return 1 - o / e

In [61]:
features = [i for i in comp_train_df.columns if i not in ['accuracy_group', 'installation_id']]
target = 'accuracy_group'
num_splits = 10
params = {
    'learning_rate': 0.007,
    'metric': 'multiclass',
    'objective': 'multiclass',
    'num_classes': 4,
    'feature_fraction': 0.75,
    "bagging_fraction": 0.8,
    "bagging_seed": 42,
}

def train_model(comp_train_df, comp_test_df):
    
    kf = KFold(n_splits=num_splits, shuffle=True)
    
    oof_pred = np.zeros((len(comp_train_df), 4))
    models = []
    
    for fold, (tr_ind, val_ind) in enumerate(kf.split(comp_train_df)):
        print(f'Fold: {fold+1}')
        x_train, x_val = comp_train_df[features].iloc[tr_ind], comp_train_df[features].iloc[val_ind]
        y_train, y_val = comp_train_df[target][tr_ind], comp_train_df[target][val_ind]
        train_set = lgb.Dataset(x_train, y_train)
        val_set = lgb.Dataset(x_val, y_val)

        model = lgb.train(params, train_set, num_boost_round = 10000, early_stopping_rounds = 100, 
                          valid_sets=[train_set, val_set], verbose_eval = 100)
        oof_pred[val_ind] = model.predict(x_val)
        models.append(model)
        
        val_crt_fold = qwk3(y_val, oof_pred[val_ind].argmax(axis = 1))
        print(f'Fold: {fold+1} quadratic weighted kappa score: {np.round(val_crt_fold,4)}')
        
    res = qwk3(comp_train_df['accuracy_group'], oof_pred.argmax(axis = 1))
    print(f'Quadratic weighted score: {np.round(res,4)}')
        
    return models

In [62]:
models = train_model(comp_train_df, comp_test_df)

Fold: 1
Training until validation scores don't improve for 100 rounds
[100]	training's multi_logloss: 0.956024	valid_1's multi_logloss: 1.00513
[200]	training's multi_logloss: 0.832317	valid_1's multi_logloss: 0.899192
[300]	training's multi_logloss: 0.763243	valid_1's multi_logloss: 0.846633
[400]	training's multi_logloss: 0.719918	valid_1's multi_logloss: 0.818297
[500]	training's multi_logloss: 0.690785	valid_1's multi_logloss: 0.804245
[600]	training's multi_logloss: 0.670045	valid_1's multi_logloss: 0.798212
[700]	training's multi_logloss: 0.654129	valid_1's multi_logloss: 0.796477
[800]	training's multi_logloss: 0.641024	valid_1's multi_logloss: 0.797507
Early stopping, best iteration is:
[716]	training's multi_logloss: 0.651836	valid_1's multi_logloss: 0.79627
Fold: 1 quadratic weighted kappa score: 0.625
Fold: 2
Training until validation scores don't improve for 100 rounds
[100]	training's multi_logloss: 0.955845	valid_1's multi_logloss: 0.986787
[200]	training's multi_logloss:

## Inference

In [20]:
def add_missing_columns(comp_train_df: pd.DataFrame, comp_test_df: pd.DataFrame):
    missing: set = set(comp_train_df.columns) - set(comp_test_df.columns)
    for col in missing:
        comp_test_df[col] = 0.
    print(f'Added missing colums: {missing}')

In [21]:
add_missing_columns(comp_train_df, comp_test_df)

Added missing colums: {'4074bac2', '0ce40006', '17ca3959', 'accuracy_group', '1b54d27f', 'ecc6157f', '611485c5', '13f56524', '003cd2ee', 'e4d32835', '29a42aea', '5dc079d8', '01ca3a3c', 'ab4ec3a4', '119b5b02', 'bfc77bd6', 'dcb1663e', '7fd1ac25', 'a8cc6fec', '2ec694de'}


In [22]:
def run_predictions(models):
    y_pred = np.zeros((len(comp_test_df), 4))
    for model in models:
        y_pred += model.predict(comp_test_df[features])
    return y_pred / num_splits

In [23]:
y_pred = run_predictions(models)

In [24]:
assert comp_test_df.shape[0] == y_pred.shape[0]

In [25]:
np.unique(y_pred.argmax(-1), return_counts=True)

(array([0, 1, 2, 3]), array([652,  41,  29, 278]))

In [26]:
def prepare_submission(comp_test_df, sample_submission_df, y_pred):
    comp_test_df = comp_test_df.reset_index()
    comp_test_df = comp_test_df[['installation_id']]
    comp_test_df['accuracy_group'] = y_pred.argmax(axis = 1)
    sample_submission_df.drop('accuracy_group', inplace = True, axis = 1)
    sample_submission_df = sample_submission_df.merge(comp_test_df, on = 'installation_id')
    sample_submission_df.to_csv('submission.csv', index = False)

In [27]:
prepare_submission(comp_test_df, sample_submission_df, y_pred)

In [28]:
!head submission.csv

installation_id,accuracy_group
00abaee7,3
01242218,3
017c5718,0
01a44906,0
01bc6cb6,0
02256298,3
0267757a,0
027e7ce5,3
02a29f99,0


In [29]:
!cat submission.csv | wc -l

1001
